In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from datetime import timedelta, datetime

In [2]:
def Parse_tag_to_data(Tag):
    data = { k:'' for k in ['#', 'Country', 'Total Cases', 'New Cases', 'Total Deaths', 'New Deaths', 'Total Recover', 
                            'New Recorver', 'Active Cases', 'Serious Critical', 'Cases/1M pop','Deaths/1M pop', 
                              'Total tests', 'Tests/1M pop', 'Population', 'Continent']}
    for key, value in zip(data, Tag.find_all('td')):
        data[key] = value.text.strip()
    return data

In [3]:
driver = webdriver.Chrome()
driver.get('https://www.worldometers.info/coronavirus/#main_table')

In [4]:
soup = BeautifulSoup(driver.page_source, 'html.parser')
driver.quit()

In [5]:
odd = soup.find_all('tr', class_='odd')
even = soup.find_all('tr', class_='even')
data = odd + even
type(data[0])

bs4.element.Tag

In [6]:
Continent = []
Country = []
for item in data:
    temp_data = Parse_tag_to_data(item)
    if temp_data['Country'] == '':
        continue
    elif 'total_row_world' in item['class']:
        temp_data['#'] = 0
        Continent.append(temp_data)
        continue
    
    temp_data['#'] = int(temp_data['#'])
    Country.append(temp_data)
Continent = [item for item in sorted(Continent, key = lambda item: item['Country'])]   
Country = [item for item in sorted(Country, key = lambda item: item['#'])] 

In [9]:
Today = []
Yesterday = []
two_day_ago = []
for i in range(0,len(Country),3):
    Today.append(Country[i])
    Yesterday.append(Country[i+1])
    two_day_ago.append(Country[i+2])

for i in range(0, len(Continent), 3):
    Today.append(Continent[i])
    Yesterday.append(Continent[i+1])
    two_day_ago.append(Continent[i+2])

In [10]:
#pd.set_option('display.max_rows', None)
df1 = pd.DataFrame.from_dict(Today)
df1.to_csv(f'Dataset/{str(datetime.today().date())}.csv', sep='\t', index=False)
df1

,#,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recover,New Recorver,Active Cases,Serious Critical,Cases/1M pop,Deaths/1M pop,Total tests,Tests/1M pop,Population,Continent
0,1,USA,"50,535,791",,"815,269",,"39,879,448",,"9,841,074","14,647","151,399","2,442","769,966,461","2,306,726","333,791,863",North America
1,2,India,"34,674,744",,"474,735",,"34,105,066","+7,678","94,943","8,944","24,777",339,"651,950,127","465,847","1,399,494,746",Asia
2,3,Brazil,"22,177,059",,"616,504",,"21,407,699",,"152,856","8,318","103,279","2,871","63,776,166","297,006","214,730,471",South America
3,4,UK,"10,660,981",,"146,135",,"9,378,165",,"1,136,681",890,"155,866","2,137","373,037,852","5,453,891","68,398,479",Europe
4,5,Russia,"9,925,806",,"286,004",,"8,637,861",,"1,001,941","2,300","67,974","1,959","229,800,000","1,573,709","146,024,415",Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,0,Europe,"77,087,330","+30,258","1,450,735",+101,"67,483,526","+59,527","8,153,069","23,061",,,,,,Europe
227,0,North America,"60,313,384","+3,304","1,208,963",+296,"48,451,025","+2,478","10,653,396","20,585",,,,,,North America
228,0,Oceania,"385,855","+1,746","4,320",,"344,962",+41,"36,573",149,,,,,,Australia/Oceania
229,0,South America,"39,160,071",+25,"1,185,465",+1,"37,227,112",,"747,494","12,799",,,,,,South America


In [11]:
df2 = pd.DataFrame.from_dict(Yesterday)
df2.to_csv(f'Dataset/{str(datetime.today().date() - timedelta(1))}.csv', sep='\t', index=False)
df2

,#,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recover,New Recorver,Active Cases,Serious Critical,Cases/1M pop,Deaths/1M pop,Total tests,Tests/1M pop,Population,Continent
0,1,USA,"50,535,791","+110,894","815,269","+1,088","39,879,448","+63,582","9,841,074","14,647","151,399","2,442","769,966,461","2,306,726","333,791,863",North America
1,2,India,"34,674,744","+8,503","474,735",+624,"34,097,388","+8,251","102,621","8,944","24,777",339,"653,243,539","466,771","1,399,494,746",Asia
2,3,Brazil,"22,177,059","+9,278","616,504",+206,"21,407,699","+8,383","152,856","8,318","103,279","2,871","63,776,166","297,006","214,730,471",South America
3,4,UK,"10,660,981","+50,867","146,135",+148,"9,378,165","+37,775","1,136,681",890,"155,866","2,137","373,037,852","5,453,891","68,398,479",Europe
4,5,Russia,"9,925,806","+30,209","286,004","+1,181","8,637,861","+35,794","1,001,941","2,300","67,974","1,959","229,800,000","1,573,709","146,024,415",Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,0,Europe,"77,057,072","+346,439","1,450,634","+3,817","67,423,999","+286,584","8,182,439","23,037",,,,,,Europe
227,0,North America,"60,310,080","+120,820","1,208,667","+1,448","48,448,547","+71,437","10,652,866","20,584",,,,,,North America
228,0,Oceania,"384,109","+1,913","4,320",+10,"344,921","+1,794","34,868",142,,,,,,Australia/Oceania
229,0,South America,"39,160,046","+19,291","1,185,464",+383,"37,227,170","+16,225","747,412","12,798",,,,,,South America


In [12]:
df3 = pd.DataFrame.from_dict(two_day_ago)
df3.to_csv(f'Dataset/{str(datetime.today().date() - timedelta(2))}.csv', sep='\t', index=False)
df3

,#,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recover,New Recorver,Active Cases,Serious Critical,Cases/1M pop,Deaths/1M pop,Total tests,Tests/1M pop,Population,Continent
0,1,USA,"50,424,897","+129,232","814,181","+1,366","39,815,866","+71,029","9,794,850","14,473","151,067","2,439","768,344,446","2,301,867","333,791,863",North America
1,2,India,"34,666,241","+9,419","474,111",+159,"34,089,137","+9,525","102,993","8,944","24,771",339,"650,660,144","464,925","1,399,494,746",Asia
2,3,Brazil,"22,167,781","+10,055","616,298",+231,"21,399,316","+13,045","152,167","8,318","103,235","2,870","63,776,166","297,006","214,730,471",South America
3,4,UK,"10,610,114","+51,060","145,987",+161,"9,340,390","+39,692","1,123,737",880,"155,122","2,134","371,694,121","5,434,245","68,398,479",Europe
4,5,Russia,"9,895,597","+30,752","284,823","+1,179","8,602,067","+36,976","1,008,707","2,300","67,767","1,951","229,200,000","1,569,601","146,024,415",Europe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
226,0,Europe,"76,710,633","+412,737","1,446,817","+4,501","67,137,415","+291,642","8,126,401","22,775",,,,,,Europe
227,0,North America,"60,189,260","+138,689","1,207,219","+1,725","48,377,110","+79,481","10,604,931","20,465",,,,,,North America
228,0,Oceania,"382,196","+1,870","4,310",+8,"343,127","+3,429","34,759",148,,,,,,Australia/Oceania
229,0,South America,"39,140,755","+19,383","1,185,081",+370,"37,210,945","+19,782","744,729","12,791",,,,,,South America
